In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
import os

In [2]:
def image_plot(item_dir, top=10):
    all_item_dirs = os.listdir(item_dir)
    item_files = [os.path.join(item_dir, file) for file in all_item_dirs[:5]]
    
    plt.figure(figsize=(12,14))
    
    for idx, img_path in enumerate(item_files):
        plt.subplot(5,5,idx+1)
        
        img=plt.imread(img_path)
        plt.tight_layout()
        plt.imshow(img, cmap='gray')

In [3]:
print(os.listdir('Garbage classification/Garbage classification'))

['glass', 'trash', 'plastic', 'cardboard', 'paper', 'metal']


In [4]:
data_path = 'Garbage classification/Garbage classification'

In [5]:
train = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, validation_split=0.1, rescale=1./255,
                           shear_range=0.1,zoom_range=0.1, width_shift_range=0.1,height_shift_range=0.1)

test = ImageDataGenerator(rescale=1./255, validation_split=0.1)

In [6]:
shape = (224, 224)
train_generator = train.flow_from_directory(data_path,target_size=shape,batch_size=32, class_mode='categorical',
                                            subset='training')

test_generator = test.flow_from_directory(data_path,target_size=shape,batch_size=32, class_mode='categorical',
                                            subset='validation')

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


In [7]:
base_model=MobileNet(input_shape=(224,224,3), weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(6,activation='softmax')(x) #final layer with softmax activation

In [8]:
model=Model(inputs=base_model.input,outputs=preds)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [9]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'])

In [10]:
model_path='mdelVnet-garbage_model.h5'
checkpoint1 = ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

In [11]:
step_size_train = train_generator.n//train_generator.batch_size
step_size_test = test_generator.n//test_generator.batch_size

In [12]:
history=model.fit_generator(train_generator, epochs=15, steps_per_epoch=step_size_train, validation_data=test_generator,
                           validation_steps=step_size_test, callbacks=callbacks_list)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
71/71 [==============================] - ETA: 0s - loss: 1.0014 - acc: 0.6496
Epoch 00001: val_acc improved from -inf to 0.32143, saving model to mdelVnet-garbage_model.h5
71/71 [==============================] - 505s 7s/step - loss: 1.0014 - acc: 0.6496 - val_loss: 3.6209 - val_acc: 0.3214
Epoch 2/15
71/71 [==============================] - ETA: 0s - loss: 0.6861 - acc: 0.7767
Epoch 00002: val_acc improved from 0.32143 to 0.35268, saving model to mdelVnet-garbage_model.h5
71/71 [==============================] - 491s 7s/step - loss: 0.6861 - acc: 0.7767 - val_loss: 4.6974 - val_acc: 0.3527
Epoch 3/15
71/71 [==============================] - ETA: 0s - loss: 0.5549 - acc: 0.8142
Epoch 00003: val_acc improved from 0.35268 to 0.45536, saving model to mdelVnet-garbage_model.h5
71/71 [==============================] - 485s 7s/step - loss: 0.5549 - acc: 0.8142 - val_loss: 2.2389 - val_acc: 0.4554
Epoch 4/1

In [13]:
model = tf.keras.models.load_model('mdelVnet-garbage_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()#saving converted model in "converted_model.tflite" file
open("model_unquant.tflite", "wb").write(tflite_model)

model1 = tf.keras.models.load_model('mdelVnet-garbage_model.h5')
converter1 = tf.lite.TFLiteConverter.from_keras_model(model1)
converter1.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter1.convert()#saving converted model in "converted_quant_model.tflite" file
open("model.tflite", "wb").write(tflite_quant_model)

5891552